In [1]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2
from analysis_toolbox import *

In [3]:
import ast
from qualityBaseline import * 
from pycocotools.coco import COCO
import matplotlib.image as mpimg
save_db_as_csv(connect=False)
img_info,object_tbl,bb_info,hit_info = load_info()

/Users/dorislee/.virtualenvs/py27dev/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [4]:
all_x_locs = bb_info["x_locs"]
all_y_locs = bb_info["y_locs"]

In [7]:
dataDir='../../coco/'
dataType='train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=15.76s)
creating index...
index created!


In [9]:
ground_truth = pd.read_csv("../../data/object_ground_truth.csv")

In [10]:
def process_raw_locs(segmentation,COCO=False):
    '''
    Given a raw string of x and y coordinates, process it
    return a list of x_locs and y_locs
    '''
    x_locs=[]
    y_locs=[]
    if COCO:
        #COCO
#         print "Process COCO"
        poly = np.array(segmentation).reshape((len(segmentation)/2, 2))
        x_locs = list(poly[:,0])
        y_locs = list(poly[:,1])
    else: 
        bbx_path,bby_path = segmentation
        x_locs = [x for x in ast.literal_eval(bbx_path) if x is not None]
        y_locs = [y for y in ast.literal_eval(bby_path) if y is not None]


    # Append the starting point again in the end to close the BB
    x_locs.append(x_locs[0])
    y_locs.append(y_locs[0])
    return x_locs,y_locs

In [11]:
bb_info['COCO Majority Vote'] = pd.Series(np.zeros(len(bb_info)), index=bb_info.index)

In [32]:
bb_info.ix[bb[0]]

object_id                                                             8
worker_id                                                             3
x_locs                [302, 264, 254, 250, 250, 250, 248, 247, 247, ...
y_locs                [124, 131, 137, 145, 152, 160, 167, 173, 188, ...
COCO Majority Vote                                                    0
Name: 0, dtype: object

In [35]:
for i in np.arange(len(img_info)):
    img_name = img_info["filename"][i]
    if 'COCO' in img_name:
        fname = "../web-app/app/static/"+img_name+".png"
        img=mpimg.imread(fname)
        width,height = get_size(fname)
        img_id = int(img_name.split('_')[-1])
        filtered_object_tbl = object_tbl[object_tbl["image_id"]==i+1]
        annIds = coco.getAnnIds(imgIds=img_id, iscrowd=None)
        anns = coco.loadAnns(annIds)
        #for oid,bbx_path,bby_path in zip(bb_info["object_id"],bb_info["x_locs"],bb_info["y_locs"]):
        for bb in bb_info.iterrows():
            oid = bb[1]["object_id"]
            bbx_path= bb[1]["x_locs"]
            bby_path= bb[1]["y_locs"]
            if int(object_tbl[object_tbl.object_id==oid].image_id) ==i+1:
                worker_x_locs,worker_y_locs= process_raw_locs([bbx_path,bby_path])
                ground_truth_match = ground_truth[ground_truth.id==str(oid)]
                COCO_id = int(ground_truth_match["COCO_annIds"])
                
                #COCO-Annotations
                for ann in anns:
                    if COCO_id==-1:
                        #No BB for this object collected by MSCOCO
                        pass
                    elif ann['id'] == COCO_id: 
#                         print COCO_id
                        for annBB in ann["segmentation"]:
                            bbg_x_locs,bbg_y_locs = process_raw_locs(annBB,COCO=True)
    #                         print ann["segmentation"]
#                             print majority_vote([worker_x_locs,bbg_x_locs],[worker_y_locs,bbg_y_locs],width,height)                
                            bb_info = bb_info.set_value(bb[0],"COCO Majority Vote",majority_vote([worker_x_locs,bbg_x_locs],[worker_y_locs,bbg_y_locs],width,height))                

In [36]:
bb_info

,object_id,worker_id,x_locs,y_locs,COCO Majority Vote
0,8,3,"[302, 264, 254, 250, 250, 250, 248, 247, 247, ...","[124, 131, 137, 145, 152, 160, 167, 173, 188, ...",0.000000
1,11,3,"[347, 383, 385, 380, 393, 414, 429, 429, 408, ...","[278, 321, 332, 341, 351, 357, 356, 338, 333, ...",1.120050
2,16,3,"[354, 353, 353, 361, 378, 390, 402, 409, 413, ...","[455, 434, 405, 381, 370, 370, 381, 393, 413, ...",1.076243
3,9,3,"[258, 258, 291, 303, 355, 349, 357, 363, 388, ...","[218, 270, 275, 289, 285, 279, 278, 283, 281, ...",1.031952
4,13,3,"[243, 181, 190, 224, 238, 247, 254, 268, 272, ...","[273, 345, 376, 398, 399, 400, 400, 404, 385, ...",1.076111
5,5,3,"[432, 437, 443, 456, 454, 474, 483, 483, 486, ...","[208, 195, 173, 168, 136, 130, 136, 142, 151, ...",0.000000
6,43,3,"[103, 94, 89, 96, 106, 118, 128, 136, 125, 137...","[432, 430, 441, 441, 446, 453, 453, 449, 476, ...",0.000000
7,4,3,"[91, 96, 93, 88, 85, 88, 108, 124, 134, 134, 1...","[335, 355, 381, 418, 444, 455, 462, 452, 442, ...",0.000000
8,37,3,"[522, 522, 517, 517, 528, 536, 544, 544, 541, ...","[225, 217, 210, 199, 192, 195, 204, 211, 218, ...",1.002413
9,3,3,"[47, 90, 97, 90, 86, 47, 52]","[319, 340, 360, 409, 422, 407, 340]",0.000000
